# TODO:
- [x] UPDATEN VAN DE bag_remove KOLOMMEN
- [x] BASE NOTEBOOK UPDATEN & MASTER NOTEBOOK HIERIN AANROEPEN (PAPERMILL)
- [ ] MACHINE LEARNING WEER LATEN WERKEN
- [ ] DOCUMENTEREN NOTEBOOKS
- [ ] DOCUMENTEREN README
- [ ] PARAMETER OPTIMALISATIE SEARCH RUNNEN

In [2]:
# Load public modules.
import os, sys
from sklearn.pipeline import Pipeline
from pathlib import Path

# Get the home dir and username.
HOME = str(Path.home())
USERNAME = os.path.basename(HOME)

# Set codebase path for old VAO.
CODEBASE_PATH_OLD = os.path.join(HOME, 'Documents/woonfraude/codebase')
sys.path.insert(1, CODEBASE_PATH_OLD)
                
# Set codebase path for new VAO.
CODEBASE_PATH_NEW = os.path.join('/data', USERNAME, 'Documents/woonfraude/codebase')
sys.path.insert(1, CODEBASE_PATH_NEW)

# Import own modules.
from datasets_oo import *
from clean_oo import *
from extract_features_oo import *

In [3]:
# Set global variables.
FORCE_DOWNLOAD = False
FORCE_DATASET_SPECIFIC_PREPROCESSING = False

# Load all datasets in memory

In [4]:
##############################
# Initialize dataset objects #
##############################

adresDataset = AdresDataset()
zakenDataset = ZakenDataset()
stadiaDataset = StadiaDataset()
personenDataset = PersonenDataset()
bagDataset = BagDataset()
hotlineDataset = HotlineDataset()

In [4]:
#######################################################################################
# Download data and perform dataset-specific pre-processing steps for dataset objects #
#######################################################################################

# Forces the downloading of new data.
if FORCE_DOWNLOAD:
    adresDataset.download(force=True)
    zakenDataset.download(force=True)
    stadiaDataset.download(force=True)
    personenDataset.download(force=True)
    bagDataset.download(force=True)
    

 # Forces the dataset specific pre-processing of the downloaded data.
if FORCE_DATASET_SPECIFIC_PREPROCESSING:
    
    # Adres dataset.
    adresDataset.load('download')
    adresDataset.extract_leegstand()
    adresDataset.enrich_with_woning_id()

    # Zaken Dataset.
    zakenDataset.load('download')
    zakenDataset.add_categories()
    zakenDataset.filter_categories()

    # Stadia dataset.
    stadiaDataset.load('download')
    stadiaDataset.add_zaak_stadium_ids()
    stadiaDataset.add_labels()

    # Personen dataset.


    # Bag dataset.
    bagDataset.download(force=True)
    bagDataset.load('download')
    bagDataset.bag_fix()

    # Hotline dataset.
    hotlineDataset.download(force=True)

In [5]:
###################
## Load datasets ##
###################
# Load datasets from cache (when download and pre-processing steps in previous block have been done).

adresDataset.load('download_leegstand_woningId')
zakenDataset.load('download_categories_filterCategories')
stadiaDataset.load('download_ids_labels')
personenDataset.load('download')
bagDataset.load('download_columnFix')
hotlineDataset.load('download')

Version 'download_leegstand_woningId' of dataset 'adres' loaded!
Version 'download_categories_filterCategories' of dataset 'zaken' loaded!
Version 'download_ids_labels' of dataset 'stadia' loaded!
Version 'download' of dataset 'personen' loaded!
Version 'download_columnFix' of dataset 'bag' loaded!
Version 'download' of dataset 'hotline' loaded!


# Clean and extract features from all datasets

In [6]:
######################################################
## Clean zaken dataset & perform feature extraction ##
######################################################

zakenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=zakenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum','einddatum', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    ),
    ('extract', FeatureExtractionTransformer(
        categorical_cols_hot=['afg_code_beh', 'beh_code', 'eigenaar', 'categorie'])
    )
    ])

zaken = zakenPipeline.fit_transform(zakenDataset.data)

Dataframe "zaken": Dropped 0 duplicates!
Dataframe "zaken": Fixed dates!
Dataframe "zaken": Cleaned out 3 dates!
Lowered strings of cols ['beh_code', 'beh_oms', 'afg_code_beh', 'afs_code', 'afs_oms', 'afg_code_afs', 'eigenaar', 'zaak_id', 'mededelingen', 'categorie'] in df zaken!
Missing values in df zaken have been imputed!
Now extracting features from column: 'afg_code_beh'.
Done!
Now extracting features from column: 'beh_code'.
Done!
Now extracting features from column: 'eigenaar'.
Done!
Now extracting features from column: 'categorie'.
Done!


In [7]:
##########################
## Clean stadia dataset ##
##########################

stadiaPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=stadiaDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['begindatum', 'peildatum', 'einddatum', 'date_created',
                          'date_modified', 'wzs_update_datumtijd'],
        clean_dates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

stadia = stadiaPipeline.fit_transform(stadiaDataset.data)

Dataframe "stadia": Dropped 0 duplicates!
Dataframe "stadia": Fixed dates!
Dataframe "stadia": Cleaned out 0 dates!
Lowered strings of cols ['afg_co', 'sta_code', 'sta_oms', 'afg_code_stad', 'afs_code', 'afs_oms', 'afg_code_afs', 'resultaat', 'mdr_code', 'user_created', 'user_modified', 'stadia_id', 'zaak_id', 'stadium_id', 'label'] in df stadia!
Missing values in df stadia have been imputed!


In [8]:
############################
## Clean personen dataset ##
############################

personenPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=personenDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['geboortedatum'],
        lower_string_columns=True)
    )])

personen = personenPipeline.fit_transform(personenDataset.data)

Dataframe "personen": Dropped 0 duplicates!
Dataframe "personen": Fixed dates!
Lowered strings of cols ['pen_type', 'gezinsverhouding', 'geslacht', 'voorletters', 'burgerlijke_staat', 'naam', 'geheim_adres', 'voorv_mnaam', 'voorv_naam', 'meisjesnaam', 'vertrekdatum_adam', 'ind_naamgebruik', 'nat_ned', 'ind_nat_ovlp', 'verblijfstatus', 'datum_einde_vblstat', 'landcode', 'user_created', 'user_modified', 'datum_begin_vblstat', 'ais_nr', 'crv_nr', 'geheim', 'in_onderzoek', 'datum_verkrijging_vreemd', 'voorletters_zdia', 'naam_zdia', 'voorv_mnaam_zdia', 'voorv_naam_zdia', 'meisjesnaam_zdia', 'nm_dia_255', 'mnm_dia_255'] in df personen!
Missing values in df personen have been imputed!


In [9]:
####################################################
## Clean BAG dataset & perform feature extraction ##
####################################################

bagPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=bagDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=[],
        drop_columns = ['indicatie_geconstateerd', 'indicatie_in_onderzoek', 'woningvoorraad'],
        lower_string_columns=True,
        impute_missing_values=True,
        impute_missing_values_mode=['status_coordinaat_code'],
#         impute_missing_values_mode=['status_coordinaat_code', 'indicatie_geconstateerd',
#                                     'indicatie_in_onderzoek', 'woningvoorraad'],
        fillna_columns={'_huisnummer_verblijfsobject': 0,
                         '_huisletter_verblijfsobject': 'None',
                         '_openbare_ruimte_naam_verblijfsobject': 'None',
                         '_huisnummer_toevoeging_verblijfsobject': 'None',
                         'type_woonobject_omschrijving': 'None',
                         'eigendomsverhouding_id': 'None',
                         'financieringswijze_id': -1,
                         'gebruik_id': -1,
                         'reden_opvoer_id': -1,
                         'status_id_verblijfsobject': -1,
                         'toegang_id': 'None'})
    ),
#     ('extract', FeatureExtractionTransformer(
#         categorical_cols_hot=['status_coordinaat_code', 'type_woonobject_omschrijving',
#                               'eigendomsverhouding_id', 'financieringswijze_id',
#                               'gebruik_id', 'ligging_id', 'reden_opvoer_id',
#                               'status_id_nummeraanduiding', 'toegang_id'])
#     )
    ])

bagDataset.data = bagPipeline.fit_transform(bagDataset.data)
bag = bagDataset.data  # For easier usage.

Dataframe "bag": Dropped 0 duplicates!
Lowered strings of cols ['id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_nummeraanduiding', 'huisnummer_toevoeging_nummeraanduiding', 'postcode', 'type', 'adres_nummer', 'vervallen_nummeraanduiding', 'hoofdadres', '_openbare_ruimte_naam_nummeraanduiding', 'bron_id_nummeraanduiding', 'ligplaats_id', 'openbare_ruimte_id', 'standplaats_id', 'status_id_nummeraanduiding', 'verblijfsobject_id', '_geom', 'id_ligplaats', 'landelijk_id_ligplaats', 'vervallen_ligplaats', 'geometrie_ligplaats', 'bron_id_ligplaats', 'status_id_ligplaats', 'id_standplaats', 'landelijk_id_standplaats', 'vervallen_standplaats', 'geometrie_standplaats', 'bron_id_standplaats', 'status_id_standplaats', 'id_verblijfsobject', 'landelijk_id_verblijfsobject', 'status_coordinaat_code', 'status_coordinaat_omschrijving', 'type_woonobject_code', 'type_woonobject_omschrijving', 'geometrie_verblijfsobject', '_huisletter_verblijfsobject', 'bron_id_verblijfsobject', 'eigend

In [10]:
###########################
## Clean hotline dataset ##
###########################

hotlinePipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=hotlineDataset.id_column,
        drop_duplicates=True,
        lower_string_columns=True,
        impute_missing_values=True)
    )])

hotlineDataset.data = hotlinePipeline.fit_transform(hotlineDataset.data)
hotline = hotlineDataset.data  # For easier usage.

Dataframe "hotline": Dropped 0 duplicates!
Lowered strings of cols ['mdw_code', 'overtreding_code', 'melder_anoniem', 'melder_naam', 'melder_emailadres', 'melder_telnr', 'situatie_schets', 'user_created', 'user_modified'] in df hotline!
Missing values in df hotline have been imputed!


In [11]:
#########################
## Clean adres dataset ##
#########################

# Hier de extract stap weghalen? Deze past waarschijnlijk beter na het combinen v/d datasets.
adresPipeline = Pipeline(steps=[
    ('clean', CleanTransformer(
        id_column=adresDataset.id_column,
        drop_duplicates=True,
        fix_date_columns=['hvv_dag_tek', 'max_vestig_dtm', 'wzs_update_datumtijd'],
        lower_string_columns=True,
        impute_missing_values=True,
        fillna_columns={'hsnr': 0, 'sttnaam': 'None', 'hsltr': 'None', 'toev': 'None'})
    )
])
#     ,
#     ('extract', FeatureExtractionTransformer(
#         categorical_cols_hot=['toev', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr'],
#         ))
#     ])

adresDataset.data = adresPipeline.fit_transform(adresDataset.data)
adres = adresDataset.data  # For easier usage.

Dataframe "adres": Dropped 0 duplicates!
Dataframe "adres": Fixed dates!
Lowered strings of cols ['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'hvv_dag_tek', 'max_vestig_dtm'] in df adres!
Missing values in df adres have been imputed!


# Combine the datasets

In [12]:
##########################
## Enrich adres dataset ##
##########################

# Enrich the adres dataset with information from the bag, personen and hotline datasets.
adresDataset.enrich_with_bag(bagDataset.data)
adresDataset.enrich_with_personen_features(personenDataset.data)
adresDataset.add_hotline_features(hotlineDataset.data)

Missing values in df adres have been imputed!
Saving version 'download_leegstand_woningId_bag' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_numme

The adres dataset is now enriched with BAG data.
Now looping over all address ids that have a link with one or more inhabitants...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
Now looping over all rows in the adres dataframe in order to add person information...
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
...done!
Saving version 'download_leegstand_woningId_bag_personen' of dataframe 'adres'.


/home/tjongstra/.local/lib/python3.6/site-packages/pandas/core/generic.py:2378: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block5_values] [items->['postcode', 'sdl_code', 'brt_code', 'pvh_cd', 'pvh_omschr', 'sbw_omschr', 'sbv_omschr', 'wzs_buurtcode_os_2015', 'wzs_buurtnaam_os_2015', 'wzs_buurtcombinatiecode_os_2015', 'wzs_buurtcombinatienaam_os_2015', 'wzs_22gebiedencode_os_2015', 'wzs_22gebiedennaam_os_2015', 'wzs_rayoncode_os_2015', 'wzs_rayonnaam_os_2015', 'wzs_stadsdeelcode_os_2015', 'wzs_stadsdeelnaam_os_2015', 'wzs_alternatieve_buurtennaam_os_2015', 'wzs_alternatieve_buurtencode_os_2015', 'wzs_geom', 'wzs_wijze_verrijking_geo', 'wzs_22gebiedencode_2015', 'wzs_22gebiedennaam_2015', 'sttnaam', 'hsltr', 'toev', 'brtcombi_naam', 'sdl_naam', 'brt_naam', 'a_dam_bag', 'landelijk_bag', 'postcode_x', 'id_nummeraanduiding', 'landelijk_id_nummeraanduiding', 'huisletter_numme

The adres dataset is now enriched with personen data.
Saving version 'download_leegstand_woningId_bag_personen_hotline' of dataframe 'adres'.
The adres dataset is now enriched with hotline data.


In [13]:
list(adresDataset.data.columns)

['adres_id',
 'straatcode',
 'postcode',
 'sdl_code',
 'brt_code',
 'pvh_cd',
 'pvh_omschr',
 'sbw_omschr',
 'sbv_omschr',
 'wzs_id',
 'wzs_buurtcode_os_2015',
 'wzs_buurtnaam_os_2015',
 'wzs_buurtcombinatiecode_os_2015',
 'wzs_buurtcombinatienaam_os_2015',
 'wzs_22gebiedencode_os_2015',
 'wzs_22gebiedennaam_os_2015',
 'wzs_rayoncode_os_2015',
 'wzs_rayonnaam_os_2015',
 'wzs_stadsdeelcode_os_2015',
 'wzs_stadsdeelnaam_os_2015',
 'wzs_alternatieve_buurtennaam_os_2015',
 'wzs_alternatieve_buurtencode_os_2015',
 'wzs_update_datumtijd',
 'wzs_geom',
 'wzs_lon',
 'wzs_lat',
 'wzs_wijze_verrijking_geo',
 'wzs_22gebiedencode_2015',
 'wzs_22gebiedennaam_2015',
 'sttnaam',
 'hsnr',
 'hsltr',
 'toev',
 'brtcombi_code',
 'brtcombi_naam',
 'sdl_naam',
 'xref',
 'yref',
 'brt_naam',
 'a_dam_bag',
 'landelijk_bag',
 'sbw_code',
 'sbv_code',
 'inwnrs',
 'kmrs',
 'hvv_dag_tek',
 'max_vestig_dtm',
 'leegstand',
 'ads_id',
 'wng_id',
 'postcode_x',
 'id_nummeraanduiding',
 'landelijk_id_nummeraanduiding

In [14]:
###############################################################################
## Remove implicit label columns and superfluous columns  from adres dataset ##
###############################################################################

adres_remove = [# Remove because cols do not exists when melding is received
                    'wzs_update_datumtijd',
                    # Remove because cols do not add extra information.
                    'kmrs',
                    'straatcode',
                    'xref',
                    'yref',
                    'postcode',
                    'wzs_buurtcode_os_2015',
                    'wzs_buurtcombinatiecode_os_2015',
                    'wzs_stadsdeelcode_os_2015',
                    'sttnaam',
                    'hvv_dag_tek', # Empty column
                    'max_vestig_dtm', # Empty column
                    'wzs_22gebiedencode_os_2015', # Empty column
                    'wzs_22gebiedennaam_os_2015', # Empty column
                    'sdl_naam',
                    'pvh_cd',
                    'sbv_code',
                    'sbw_code',
                    'wzs_wijze_verrijking_geo',
                    'wzs_22gebiedencode_2015',
                    'brt_naam',
                    'wzs_buurtnaam_os_2015',
                    'wzs_buurtcombinatienaam_os_2015',
                    'wzs_rayonnaam_os_2015',
                    'wzs_rayoncode_os_2015',
                    'wzs_stadsdeelnaam_os_2015',
                    'wzs_alternatieve_buurtennaam_os_2015',
                    'wzs_alternatieve_buurtencode_os_2015',
                    'hsltr',
                    'wzs_geom',
                    'brt_code',
                    'brtcombi_code',
                    'brtcombi_naam',
                    'sdl_code',
                    'wzs_22gebiedennaam_2015',
                    'wzs_id',
                    'a_dam_bag',
                    'landelijk_bag']

bag_remove = ['einde_geldigheid',               # Only 2 entries in column.
              'verhuurbare_eenheden',           # Only ~2k entries in column.
              'geometrie_ligplaats',            # Needs a lot of processing before being useful.
#               'bron_id',                      # Only 2 entries in column.
              'bron_id_verblijfsobject',        # Only 2 entries in column.
              'locatie_ingang_id',              # Only 2 entries in column.
              'reden_afvoer_id',                # Only a few entries in column.
              '_gebiedsgerichtwerken_id',       # Superfluous (gebied).
              '_grootstedelijkgebied_id',       # Superfluous (grootstedelijkgebied).
              'buurt_id',                       # Superfluous (buurt).
              # ONDERSTAANDE 4 KOLOMMEN KONDEN EERDER NIET WEG IVM MATCH MET ADRES DATAFRAME.
              # DEZE MOETEN NU WEL WEG, DAAROM WORDT NU HIER ALLES WEGGEHAALD.
              '_openbare_ruimte_naam_nummeraanduiding',          # Superfluous (straatnaam).
#               '_huisnummer_nummeraanduiding',                    # Superfluous (huisnummer).
#               '_huisletter_nummeraanduiding',                    # Superfluous (huisletter).
#               '_huisnummer_toevoeging_nummeraanduiding',         # Superfluous (huisnummer toevoeging).
#               'vervallen',                      # Superfluous (all values in col are equal).
              'vervallen_nummeraanduiding',
              'vervallen_ligplaats',
              'vervallen_standplaats',
              'vervallen_verblijfsobject',
              'document_mutatie',               # Not available at time of signal.
#               'date_modified',                  # Not available at time of signal.
              'date_modified_nummeraanduiding', # Not available at time of signal.
              'document_nummer',                # Not needed? (Swaan?)
              'status_coordinaat_omschrijving', # Not needed? (Swaan?)
              'type_woonobject_code',           # Not needed? (Swaan?)
              'id_ligplaats',                   # Not needed.
              'landelijk_id_ligplaats',         # Not needed.
              'id_standplaats',                 # Not needed.
              'landelijk_id_standplaats',       # Not needed.
              'id_verblijfsobject',             # Not needed.
              'landelijk_id_verblijfsobject',   # Not needed.
              ]

# Remove adres_id, since this is not a feature we want our algorihtm to try and learn from.
# adresDataset.data.drop(columns=adres_remove + bag_remove + ['adres_id'], inplace=True)
adresDataset.data.drop(columns=adres_remove + bag_remove, inplace=True)

In [15]:
####################################################
## Merge the adres dataset onto the zaken dataset ##
####################################################

df = zakenDataset.data.merge(adresDataset.data, on='adres_id', how='left')
df.drop(columns=['adres_id'])

,wvs_nr,begindatum,einddatum,beh_code,beh_oms,afg_code_beh,afs_code,afs_oms,afg_code_afs,kamer_aantal,...,percentage_gezinsverhouding_3,gezinsverhouding_4,percentage_gezinsverhouding_4,gezinsverhouding_5,percentage_gezinsverhouding_5,gezinsverhouding_6,percentage_gezinsverhouding_6,gezinsverhouding_7,percentage_gezinsverhouding_7,aantal_hotline_meldingen
0,4.0,2011-10-12,2014-06-24 00:00:00.000000000,hsbbpart,bestuurlijke boete part,hanzlt,povvheind,op verzoek van handhaving beeindigd,vtbd,1.000000,...,0.000000,0.0,0.0,0.0,0.000000,2.0,1.000000,0.0,0.0,0.0
1,1.0,2011-10-25,2011-11-22 00:00:00.000000000,zkl_0064,zoekl.project b64,hanzlt,zzl_eind,zoeklicht beeindigd,hanzlt,2.000000,...,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000,0.0,0.0,1.0
2,3.0,2011-10-25,2011-11-25 00:00:00.000000000,zkl_0064,zoekl.project b64,hanzlt,zzl_eind,zoeklicht beeindigd,hanzlt,2.000000,...,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000,0.0,0.0,0.0
3,1.0,2011-10-25,2016-12-31 00:00:00.000000000,zkl_0064,zoekl.project b64,hanzlt,bulk_afs,bulkafsluiting,vtbd,3.000000,...,0.166667,0.0,0.0,1.0,0.166667,4.0,0.666667,0.0,0.0,0.0
4,2.0,2011-11-14,2011-12-14 00:00:00.000000000,h_pgl,project green light,hanzlt,pvkoop,woning is verkocht,vtbd,2.000000,...,0.000000,0.0,0.0,1.0,0.333333,1.0,0.333333,0.0,0.0,0.0
5,3.0,2011-11-10,2011-11-30 00:00:00.000000000,h_pgl,project green light,hanzlt,pbdbew_ret,hoofdbewoner retour op woning,vtbd,3.000000,...,0.200000,0.0,0.0,2.0,0.200000,4.0,0.400000,0.0,0.0,0.0
6,1.0,2011-11-10,2011-12-01 00:00:00.000000000,h_pgl,project green light,hanzlt,pbhpg,woning boven hpg,vtbd,2.000000,...,0.000000,0.0,0.0,0.0,0.000000,3.0,1.000000,0.0,0.0,1.0
7,1.0,2011-12-19,2012-03-20 00:00:00.000000000,h_weststrm,westerstorm,hanzlt,pbdbew_ret,hoofdbewoner retour op woning,vtbd,3.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
8,1.0,2011-12-19,2012-01-24 00:00:00.000000000,h_weststrm,westerstorm,hanzlt,p_over,overige,vtbd,1.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
9,2.0,2012-01-24,2014-06-02 00:00:00.000000000,h_ypsilon2,ypsilon 2,hanzlt,pvkoop,woning is verkocht,vtbd,2.000000,...,0.000000,0.0,0.0,0.0,0.000000,2.0,1.000000,0.0,0.0,0.0


In [16]:
# Als deze head zonder errors hierboven geprint wordt, dan werkt alles tot nu toe mooi.
# Let wel dat de --EXTRACT-- stap nog niet toegepast is op de ADRES-kolommen en de BAG-kolommen.
# (Dit wel doen zou er voor hebben gezorgd dat de adres dataset niet meer met de BAG dataset verrijkt kon worden).
df.head()

,adres_id,wvs_nr,begindatum,einddatum,beh_code,beh_oms,afg_code_beh,afs_code,afs_oms,afg_code_afs,...,percentage_gezinsverhouding_3,gezinsverhouding_4,percentage_gezinsverhouding_4,gezinsverhouding_5,percentage_gezinsverhouding_5,gezinsverhouding_6,percentage_gezinsverhouding_6,gezinsverhouding_7,percentage_gezinsverhouding_7,aantal_hotline_meldingen
0,147394.0,4.0,2011-10-12,2014-06-24,hsbbpart,bestuurlijke boete part,hanzlt,povvheind,op verzoek van handhaving beeindigd,vtbd,...,0.000000,0.0,0.0,0.0,0.000000,2.0,1.000000,0.0,0.0,0.0
1,308078.0,1.0,2011-10-25,2011-11-22,zkl_0064,zoekl.project b64,hanzlt,zzl_eind,zoeklicht beeindigd,hanzlt,...,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000,0.0,0.0,1.0
2,264274.0,3.0,2011-10-25,2011-11-25,zkl_0064,zoekl.project b64,hanzlt,zzl_eind,zoeklicht beeindigd,hanzlt,...,0.000000,0.0,0.0,0.0,0.000000,1.0,1.000000,0.0,0.0,0.0
3,238092.0,1.0,2011-10-25,2016-12-31,zkl_0064,zoekl.project b64,hanzlt,bulk_afs,bulkafsluiting,vtbd,...,0.166667,0.0,0.0,1.0,0.166667,4.0,0.666667,0.0,0.0,0.0
4,286503.0,2.0,2011-11-14,2011-12-14,h_pgl,project green light,hanzlt,pvkoop,woning is verkocht,vtbd,...,0.000000,0.0,0.0,1.0,0.333333,1.0,0.333333,0.0,0.0,0.0
